In [ ]:
!pip install optimum[onnxruntime] onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers accelerate

In [ ]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from transformers import AutoTokenizer

model_id = "Prathyusha101/led-large-16384-arxiv"

# This will automatically convert and export to ONNX
model = ORTModelForSeq2SeqLM.from_pretrained(model_id, export=True)
model.save_pretrained("led_onnx")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

ValueError: Trying to export a led model, that is a custom or unsupported architecture, but no custom onnx configuration was passed as `custom_onnx_configs`. Please refer to https://huggingface.co/docs/optimum/main/en/exporters/onnx/usage_guides/export_a_model#custom-export-of-transformers-models for an example on how to export custom models. Please open an issue at https://github.com/huggingface/optimum/issues if you would like the model type led to be supported natively in the ONNX export.

In [ ]:
import torch
from transformers import LEDTokenizer, LEDForConditionalGeneration

model_id = "Prathyusha101/led-large-16384-arxiv"

# Load tokenizer and model
tokenizer = LEDTokenizer.from_pretrained(model_id)
model = LEDForConditionalGeneration.from_pretrained(model_id)
model.eval()

# Prepare dummy input (keep it small for export)
sample_text = "This is a long document used for testing the ONNX export."
inputs = tokenizer(sample_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

# Add global attention mask (LED requires it)
inputs["global_attention_mask"] = torch.zeros_like(inputs["input_ids"])
inputs["global_attention_mask"][:, 0] = 1  # CLS token gets global attention

# Set dynamic axes so ONNX model can handle different batch sizes and sequence lengths
dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence_length'},
    'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
    'global_attention_mask': {0: 'batch_size', 1: 'sequence_length'},
    'decoder_input_ids': {0: 'batch_size', 1: 'target_sequence_length'},
    'logits': {0: 'batch_size', 1: 'target_sequence_length'}
}

# Run export
torch.onnx.export(
    model,
    args=(
        inputs["input_ids"],
        inputs["attention_mask"],
        inputs["global_attention_mask"],
        inputs["input_ids"]  # decoder_input_ids as placeholder
    ),
    f="led_model.onnx",
    input_names=["input_ids", "attention_mask", "global_attention_mask", "decoder_input_ids"],
    output_names=["logits"],
    dynamic_axes=dynamic_axes,
    opset_version=17,
    do_constant_folding=True,
    use_external_data_format=True  # Needed if model > 2GB
)

print("✅ Export complete! ONNX model saved as led_model.onnx")


/usr/local/lib/python3.11/dist-packages/torch/onnx/utils.py:657: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/usr/local/lib/python3.11/dist-packages/torch/onnx/utils.py:1127: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_graph_shape_type_inference(


✅ Export complete! ONNX model saved as led_model.onnx


In [ ]:
import os

file_path = "led_model.onnx"  # Replace with your actual file path
size_bytes = os.path.getsize(file_path)
size_mb = size_bytes / (1024 * 1024)
print(f"Model size: {size_mb:.2f} MB")


Model size: 1837.67 MB


In [ ]:
z!pip install --upgrade huggingface_hub

In [ ]:
!pip install onnx_shrink_ray


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.7 MB/s eta 0:00:00


In [ ]:
!onnx-shrink-ray led_model.onnx led_model_shrunk.onnx

/bin/bash: line 1: onnx-shrink-ray: command not found


In [ ]:
!python -m onnx_shrink_ray led_model.onnx led_model_shrunk.onnx

/usr/bin/python3: No module named onnx_shrink_ray.__main__; 'onnx_shrink_ray' is a package and cannot be directly executed


In [ ]:
!python -m onnx_shrink_ray.shrink led_model.onnx

/usr/local/lib/python3.11/dist-packages/onnx_shrink_ray/shrink.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  inverse_range = 1.0 / range_val
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/onnx_shrink_ray/shrink.py", line 321, in <module>
    new_model = quantize_weights(original_model, float_quantization=float_quantization, float_levels=args.float_levels)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/onnx_shrink_ray/shrink.py", line 162, in quantize_weights
    quantize_weights_for_graph(graph, graph, already_processed, min_elements, float_quantization, float_levels)
  File "/usr/local/lib/python3.11/dist-packages/onnx_shrink_ray/shrink.py", line 129, in quantize_weights_for_graph
    quantize_tensor(name

In [ ]:
!pip install onnx-neural-compressor "onnxruntime>=1.17.0" onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
model

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 1024)
      (layers): ModuleList(
        (0-11): 12 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
              (value_global): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (output): Linear(in_features=1024, out_features=1024, bias=True

In [ ]:
from onnx_neural_compressor.quantization import matmul_nbits_quantizer

algo_config = matmul_nbits_quantizer.RTNWeightOnlyQuantConfig()
quant = matmul_nbits_quantizer.MatMulNBitsQuantizer(
    model,
    n_bits=4,
    block_size=32,
    is_symmetric=True,
    algo_config=algo_config,
)
quant.process()
best_model = quant.model

AttributeError: 'LEDForConditionalGeneration' object has no attribute 'graph'

In [ ]:
import onnx
from onnx_neural_compressor.quantization import matmul_nbits_quantizer

# Load your existing ONNX model
onnx_model = onnx.load("led_model.onnx")  # Replace with your actual ONNX file path

# Configure the quantizer
algo_config = matmul_nbits_quantizer.RTNWeightOnlyQuantConfig()
quant = matmul_nbits_quantizer.MatMulNBitsQuantizer(
    onnx_model,  # Using your pre-existing ONNX model
    n_bits=4,
    block_size=32,
    is_symmetric=True,
    algo_config=algo_config,
)

# Run the quantization process
quant.process()
best_model = quant.model

# Save the quantized model
onnx.save(best_model, "quantized_model.onnx")

2025-04-14 21:16:47 [INFO][matmul_nbits_quantizer.py:193] Start graph optimization...
2025-04-14 21:17:02 [INFO][matmul_nbits_quantizer.py:206] Graph optimization done.
2025-04-14 21:17:02 [INFO][matmul_nbits_quantizer.py:208] start to quantize model with RTN algorithm...


Progress: [####################] 100.00%

2025-04-14 21:17:16 [INFO][utility.py:160] |******Mixed Precision Statistics******|
2025-04-14 21:17:16 [INFO][utility.py:162] +---------+-------+-----------+--------+
2025-04-14 21:17:16 [INFO][utility.py:162] | Op Type | Total |  A32W4G32 |  FP32  |
2025-04-14 21:17:16 [INFO][utility.py:162] +---------+-------+-----------+--------+
2025-04-14 21:17:16 [INFO][utility.py:162] |  MatMul |  241  |    193    |   48   |
2025-04-14 21:17:16 [INFO][utility.py:162] +---------+-------+-----------+--------+
2025-04-14 21:17:16 [INFO][matmul_nbits_quantizer.py:215] complete quantization of model with RTN algorithm.


In [ ]:
import onnxruntime as ort
import numpy as np
from transformers import AutoTokenizer
import time

In [ ]:


# Load the tokenizer that matches your model
tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")  # Replace with your model's tokenizer

# Path to your quantized model
quantized_model_path = "quantized_model.onnx"

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:


# Create an ONNX Runtime inference session with optimizations
session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session = ort.InferenceSession(quantized_model_path, session_options)

In [ ]:


# Get input and output names
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]
print(f"Model inputs: {input_names}")
print(f"Model outputs: {output_names}")

Model inputs: ['input_ids', 'attention_mask', 'global_attention_mask', 'decoder_input_ids']
Model outputs: ['logits', 'onnx::Reshape_36584', 'onnx::Reshape_36589', 'onnx::Reshape_36713', 'onnx::Reshape_36718', 'onnx::Reshape_36858', 'onnx::Reshape_36863', 'onnx::Reshape_36987', 'onnx::Reshape_36992', 'onnx::Reshape_37132', 'onnx::Reshape_37137', 'onnx::Reshape_37261', 'onnx::Reshape_37266', 'onnx::Reshape_37406', 'onnx::Reshape_37411', 'onnx::Reshape_37535', 'onnx::Reshape_37540', 'onnx::Reshape_37680', 'onnx::Reshape_37685', 'onnx::Reshape_37809', 'onnx::Reshape_37814', 'onnx::Reshape_37954', 'onnx::Reshape_37959', 'onnx::Reshape_38083', 'onnx::Reshape_38088', 'onnx::Reshape_38228', 'onnx::Reshape_38233', 'onnx::Reshape_38357', 'onnx::Reshape_38362', 'onnx::Reshape_38502', 'onnx::Reshape_38507', 'onnx::Reshape_38631', 'onnx::Reshape_38636', 'onnx::Reshape_38776', 'onnx::Reshape_38781', 'onnx::Reshape_38905', 'onnx::Reshape_38910', 'onnx::Reshape_39050', 'onnx::Reshape_39055', 'onnx::R

In [ ]:
import pandas as pd
df = pd.read_csv("articl_2.csv")
text = df["article"][0]

In [ ]:
import numpy as np
import onnxruntime as ort

In [ ]:
def run_seq2seq_inference(model_path, input_text, tokenizer):
    # Load the ONNX model
    session = ort.InferenceSession(model_path)

    # Get model input and output names
    input_names = [input.name for input in session.get_inputs()]
    output_names = [output.name for output in session.get_outputs()]

    # Tokenize input text
    encoded_input = tokenizer(input_text, return_tensors="np")

    # Create input dictionary with all required inputs
    onnx_inputs = {}

    # Add the inputs we have from the tokenizer
    for name in encoded_input:
        if name in input_names:
            onnx_inputs[name] = encoded_input[name]

    # Handle the missing required inputs that caused the error

    # For global_attention_mask (typically used in LED/Longformer models)
    if 'global_attention_mask' in input_names and 'global_attention_mask' not in onnx_inputs:
        # Create global attention mask (typically all zeros, with 1s for tokens that need global attention)
        # For most cases, you might want to set global attention to the first token only
        global_attention_mask = np.zeros_like(encoded_input['input_ids'])
        global_attention_mask[:, 0] = 1  # Set global attention to the first token
        onnx_inputs['global_attention_mask'] = global_attention_mask

    # For decoder_input_ids (used in seq2seq models)
    if 'decoder_input_ids' in input_names and 'decoder_input_ids' not in onnx_inputs:
        # Initialize decoder input ids with the decoder start token
        decoder_start_token_id = tokenizer.pad_token_id  # This might need to be a different token depending on your model
        decoder_input_ids = np.array([[decoder_start_token_id]], dtype=np.int64)
        onnx_inputs['decoder_input_ids'] = decoder_input_ids

    # Add other potential required inputs as needed
    # For example, for encoder-decoder models, you might need:
    input_shape = encoded_input['input_ids'].shape

    if 'decoder_attention_mask' in input_names and 'decoder_attention_mask' not in onnx_inputs:
        decoder_attention_mask = np.ones((input_shape[0], 1), dtype=np.int64)
        onnx_inputs['decoder_attention_mask'] = decoder_attention_mask

    # Check if all required inputs are provided
    missing_inputs = [name for name in input_names if name not in onnx_inputs]
    if missing_inputs:
        raise ValueError(f"Still missing required inputs: {missing_inputs}")

    # Run inference
    outputs = session.run(output_names, onnx_inputs)

    # Decode the output tokens to text
    if isinstance(outputs[0], np.ndarray):
        output_text = tokenizer.decode(outputs[0][0], skip_special_tokens=True)
    else:
        # Handle other output formats as needed
        output_text = "Output format not handled, check model outputs"

    return output_text


In [ ]:
# Example usage with a Hugging Face tokenizer
def inference_example():
    from transformers import AutoTokenizer

    # Load appropriate tokenizer for your model
    tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

    # Path to your quantized ONNX model
    model_path = "quantized_model.onnx"

    # Input text for inference
    input_text = text

    # Run inference
    output = run_seq2seq_inference(model_path, input_text, tokenizer)
    print(f"Model output: {output}")

In [ ]:
inference_example()

RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Add node. Name:'/led/decoder/Add_3' Status Message: /onnxruntime_src/onnxruntime/core/providers/cpu/math/element_wise_ops.h:560 void onnxruntime::BroadcastIterator::Append(ptrdiff_t, ptrdiff_t) axis == 1 || axis == largest was false. Attempting to broadcast an axis by a dimension other than 1. 1024 by 3593


In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import onnxruntime as ort
import torch

def generate_answer_onnx(batch, model_path, tokenizer):
    # Similar tokenization as your original code
    inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=16384, return_tensors="np", truncation=True)
    input_ids = inputs_dict.input_ids
    attention_mask = inputs_dict.attention_mask

    # Create global attention mask just like your original code
    global_attention_mask = np.zeros_like(attention_mask)
    # Put global attention on <s> token
    global_attention_mask[:, 0] = 1

    # Load ONNX model
    session_options = ort.SessionOptions()
    session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
    session = ort.InferenceSession(model_path, session_options, providers=providers)

    # Get input and output names
    input_names = [input.name for input in session.get_inputs()]
    output_names = [output.name for output in session.get_outputs()]

    print("Required inputs:", input_names)

    # For LED models, we need to provide decoder inputs for the first step
    # The original model.generate() handles this automatically
    decoder_input_ids = np.array([[tokenizer.eos_token_id]] * input_ids.shape[0], dtype=np.int64)

    # Prepare inputs based on model requirements
    onnx_inputs = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'global_attention_mask': global_attention_mask
    }

    # Add decoder inputs if they're required
    if 'decoder_input_ids' in input_names:
        onnx_inputs['decoder_input_ids'] = decoder_input_ids

    if 'decoder_attention_mask' in input_names:
        decoder_attention_mask = np.ones_like(decoder_input_ids)
        onnx_inputs['decoder_attention_mask'] = decoder_attention_mask

    # Print shapes for debugging
    for k, v in onnx_inputs.items():
        print(f"{k} shape: {v.shape}")

    # For sequence generation, we need to implement a loop similar to model.generate()
    # This is a simplified version - the real generate() has many more parameters

    # If your ONNX model is exported to directly handle generation:
    try:
        # Some ONNX exports include the generation logic
        outputs = session.run(output_names, onnx_inputs)
        predicted_ids = outputs[0]  # Adjust based on your model's output format

    except Exception as e:
        print(f"Direct generation failed: {e}")
        print("Your ONNX model probably doesn't have generation logic built-in.")
        print("Try re-exporting with generation logic or implement token-by-token generation.")

        # If you need to implement token-by-token generation similar to model.generate():
        # This would require a complex loop similar to HuggingFace's generation logic
        # which is quite extensive

        predicted_ids = np.zeros((input_ids.shape[0], 512), dtype=np.int64)
        # This would need to be replaced with actual token-by-token generation

    # Decode the output ids to text
    batch["predicted_abstract"] = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
    return batch

In [ ]:
generate_answer_onnx([text], "quantized_model.onnx", tokenizer)

NameError: name 'text' is not defined